### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torch.utils.data as data
import matplotlib.pyplot as plt
import IPython.display
from IPython.display import Audio
import torch.optim as optim
from types import SimpleNamespace
import scipy.signal as sc
import time
from sklearn.metrics import f1_score
import pickle

from trainDataset import TrainDataset
from testDataset import TestDataset
#from trainDatasetNew import TrainDatasetNew
#from testDatasetNew import TestDatasetNew
from validation_split import get_dataloaders
from math_utils import logMagStft, ffts
from SpectrogramCNN import SpectrogramCNN
from train_utils import train, test, save_output, save_geometric_mean_predictions
from evaluation_utils import get_mean_F1, get_loss
from MulitScale1DResNet import MSResNet
from LSTM import LSTM
from FFTMulitScale1DResNet import FFTMSResNet
from SpectralResNet import SpectralResNet18, SpectralResNet34
from scipy.signal import hilbert
from FeatureFNN import FeatureFNN
from MelRawCombined import CNN1D, CNN2D

### Parameters

In [2]:
validation_split = .1
do_plots = False
load_model = True
args = SimpleNamespace(batch_size=4, test_batch_size=4, epochs=30,
                       lr=0.0001, momentum=0.9, seed=1, log_interval=200, weight_decay=0.0001,
                      net = SpectralResNet34) #SpectrogramCNN, MSResNet, SpectralResNet34, FFTMSResNet, FeatureFNN, CNN1D, CNN2D 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not torch.cuda.is_available(): # adapt those paths on other machine
    print('no cuda')
    path_train = './../data/train-small/'
    path_test =  './../data/test/kaggle-test/'
else:
    print('with cuda')
    path_train = './../data/kaggle-train/'
    path_test =  './../data/kaggle-test/'
    
path_model = 'models/model.pt'
path_submission = 'submissions/'
    
sample_rate = 16000
nmbr_classes = 10

with cuda


### Original Dataset

In [3]:
# todo add in the classes the features and the fft data

toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

trainDataset = TrainDataset(path_train, transform=toFloat)
print(len(trainDataset))

testDataset = TestDataset(path_test, transform=toFloat)
print(len(testDataset))

269776
4096


In [4]:
input_size = len(trainDataset[0][0])
print('input size: ',input_size)

input size:  64000


### Look at Original Data

In [5]:
if do_plots:
    # how many instruments are there?
    dummy_count = np.zeros(20)

    for sample in trainDataset:
        dummy_count[sample[1]] += 1

    labels_count = []
    for elem in dummy_count:
        if elem != 0:
            labels_count.append(elem)

    print(labels_count)

In [6]:
if do_plots:
    nmbr_classes = len(labels_count)
    print('nmbr_classes: ', nmbr_classes)

In [7]:
if do_plots:
    plt.plot(labels_count, '*')

In [8]:
if do_plots:
    # plot one of each

    done = np.zeros(nmbr_classes)
    examples = []

    for sample in trainDataset:
        if done[sample[1]] == 0:
            examples.append(sample)
            done[sample[1]] = 1

In [9]:
plt.rcParams["figure.figsize"] = (20,10)

In [10]:
#time series

if do_plots:

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        plt.plot(examples[i][0])

In [11]:
#envelope

if do_plots:
    
    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        env = torch.from_numpy(np.abs(hilbert(examples[i][0]))).view(1,1,-1)
        env = m(env)
        plt.plot(env[0,0].numpy())
        
    plt.show()

In [12]:
# fft

if do_plots:

    m = nn.MaxPool1d(50)

    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        fft = torch.from_numpy(ffts(examples[i][0])).view(1,1,-1)
        fft = m(fft)
        plt.plot(fft[0,0].numpy())
        
    plt.show()

In [13]:
# melspec

if do_plots:
    n_fft = 510
    for i in range(len(examples)):
        plt.subplot(4,3,i+1)
        spectrogram = logMagStft(examples[i][0], 16000, n_fft)
        plt.imshow(spectrogram)
        
    plt.show()

In [14]:
if do_plots:

    for sample in examples:
        display(Audio(sample[0], rate=sample_rate))

### Dataloaders

In [15]:
# validation split is done here

train_loader, validation_loader, indices = get_dataloaders(trainDataset, 
                                                  batch_size = args.batch_size, 
                                                  validation_split = validation_split, 
                                                  shuffle_dataset = True, 
                                                  random_seed = None)#,
                                                  #indices = indices[0])

for samples, instrument_family_target in train_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break
        
for samples, instrument_family_target in validation_loader:
        print(samples.shape, instrument_family_target.shape,
              instrument_family_target.data)
        print(torch.min(samples), torch.max(samples))
        print(trainDataset.transformInstrumentsFamilyToString(instrument_family_target.data))
        break

torch.Size([4, 64000]) torch.Size([4]) tensor([3, 4, 5, 0])
tensor(-0.9380, dtype=torch.float64) tensor(0.9464, dtype=torch.float64)
['guitar' 'keyboard' 'mallet' 'bass']
torch.Size([4, 64000]) torch.Size([4]) tensor([2, 8, 9, 0])
tensor(-0.9466, dtype=torch.float64) tensor(0.9466, dtype=torch.float64)
['flute' 'string' 'vocal' 'bass']


In [16]:
# save indices
with open(path_submission + "indices.txt", "wb") as fp:
    pickle.dump([indices], fp)

In [17]:
test_loader = data.DataLoader(testDataset, batch_size=args.batch_size, shuffle=False) #!!! shuffle should be false
for samples in test_loader:
        print(samples[0].shape)
        print(torch.min(samples[0]), torch.max(samples[0]))
        break

torch.Size([4, 64000])
tensor(-0.8568, dtype=torch.float64) tensor(0.9467, dtype=torch.float64)


### Main

In [18]:
if (args.net == SpectrogramCNN) or (args.net == SpectralResNet18) \
    or (args.net == FeatureFNN) or (args.net == CNN1D) \
    or (args.net == CNN2D) or (args.net == SpectralResNet34):
    model = args.net(device).to(device)

In [19]:
if (args.net == MSResNet) or (args.net == FFTMSResNet):
    model = args.net(1, device).to(device)

In [20]:
if args.net == LSTM:
    model = args.net(device, input_size = 252, hidden_size = 512, num_layers = 1, num_classes = 10).to(device)

In [21]:
print(model)

SpectralResNet(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

In [22]:
# Main
#optimizer = optim.Adam(model.parameters(), lr=args.lr)

optimizer = optim.SGD(model.parameters(), lr= args.lr, momentum= args.momentum, weight_decay= args.weight_decay)

info = {'highest F1' : 0,
        'lowest loss' : 100,
        'saved epoch' : 10}

In [13]:
for epoch in range(1, args.epochs + 1):
    
    if epoch%10 == 0:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] / 10
            print('New Learning Rate: ', param_group['lr'])
    
    train(args, model, device, train_loader, optimizer, epoch, start_time = time.time())
    #f1 = get_mean_F1(model, validation_loader)
    loss_epoch, mean_f1 = get_loss(model, validation_loader, device)
    #print('after epoch {} got f1 score of {}'.format(epoch , f1))
    print('after epoch {} got loss of {} and f1 of {}'.format(epoch , loss_epoch, mean_f1))
    #if f1 > info['highest F1']:
    #    info['highest F1'] = np.copy(f1)
    #    info['saved epoch'] = epoch 
    #    test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
    #    torch.save(model, path_model)
    #    print('currently best model --> saved')
    if loss_epoch < info['lowest loss']:
        info['lowest loss'] = np.copy(loss_epoch)
        info['highest F1'] = np.copy(mean_f1)
        info['saved epoch'] = epoch 
        test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)
        torch.save(model, path_model)
        print('currently best model --> saved')    
        
print(info)

Train Epoch: 11 [0/269776 (0%)]	Loss: 0.025400	F1: 1.0000	Runtime: 0.1
Train Epoch: 11 [1600/269776 (1%)]	Loss: 0.146281	F1: 0.8750	Runtime: 10.2
Train Epoch: 11 [3200/269776 (1%)]	Loss: 0.005510	F1: 1.0000	Runtime: 20.6
Train Epoch: 11 [4800/269776 (2%)]	Loss: 0.042831	F1: 1.0000	Runtime: 30.7
Train Epoch: 11 [6400/269776 (3%)]	Loss: 0.078576	F1: 1.0000	Runtime: 40.7
Train Epoch: 11 [8000/269776 (4%)]	Loss: 0.022331	F1: 1.0000	Runtime: 50.8
Train Epoch: 11 [9600/269776 (4%)]	Loss: 0.002619	F1: 1.0000	Runtime: 60.9
Train Epoch: 11 [11200/269776 (5%)]	Loss: 0.003691	F1: 1.0000	Runtime: 71.0
Train Epoch: 11 [12800/269776 (6%)]	Loss: 0.203246	F1: 0.8750	Runtime: 81.0
Train Epoch: 11 [14400/269776 (7%)]	Loss: 0.069675	F1: 1.0000	Runtime: 91.1
Train Epoch: 11 [16000/269776 (7%)]	Loss: 0.042838	F1: 1.0000	Runtime: 101.2
Train Epoch: 11 [17600/269776 (8%)]	Loss: 0.017984	F1: 1.0000	Runtime: 111.4
Train Epoch: 11 [19200/269776 (9%)]	Loss: 0.064927	F1: 1.0000	Runtime: 121.5
Train Epoch: 11 [208

Train Epoch: 11 [168000/269776 (78%)]	Loss: 0.022066	F1: 1.0000	Runtime: 1083.7
Train Epoch: 11 [169600/269776 (79%)]	Loss: 0.650643	F1: 0.7500	Runtime: 1094.3
Train Epoch: 11 [171200/269776 (79%)]	Loss: 0.001673	F1: 1.0000	Runtime: 1104.8
Train Epoch: 11 [172800/269776 (80%)]	Loss: 0.021054	F1: 1.0000	Runtime: 1115.6
Train Epoch: 11 [174400/269776 (81%)]	Loss: 0.007133	F1: 1.0000	Runtime: 1126.1
Train Epoch: 11 [176000/269776 (82%)]	Loss: 0.027967	F1: 1.0000	Runtime: 1136.7
Train Epoch: 11 [177600/269776 (82%)]	Loss: 0.175184	F1: 0.8750	Runtime: 1147.3
Train Epoch: 11 [179200/269776 (83%)]	Loss: 0.011791	F1: 1.0000	Runtime: 1157.8
Train Epoch: 11 [180800/269776 (84%)]	Loss: 0.014817	F1: 1.0000	Runtime: 1168.3
Train Epoch: 11 [182400/269776 (85%)]	Loss: 0.006008	F1: 1.0000	Runtime: 1178.8
Train Epoch: 11 [184000/269776 (85%)]	Loss: 0.002391	F1: 1.0000	Runtime: 1189.4
Train Epoch: 11 [185600/269776 (86%)]	Loss: 0.005332	F1: 1.0000	Runtime: 1200.0
Train Epoch: 11 [187200/269776 (87%)]	Lo

Train Epoch: 12 [118400/269776 (55%)]	Loss: 0.088359	F1: 0.8750	Runtime: 755.5
Train Epoch: 12 [120000/269776 (56%)]	Loss: 0.073685	F1: 1.0000	Runtime: 765.8
Train Epoch: 12 [121600/269776 (56%)]	Loss: 0.022831	F1: 1.0000	Runtime: 776.2
Train Epoch: 12 [123200/269776 (57%)]	Loss: 0.012181	F1: 1.0000	Runtime: 786.5
Train Epoch: 12 [124800/269776 (58%)]	Loss: 0.001524	F1: 1.0000	Runtime: 797.0
Train Epoch: 12 [126400/269776 (59%)]	Loss: 0.252837	F1: 0.8750	Runtime: 807.4
Train Epoch: 12 [128000/269776 (59%)]	Loss: 0.018969	F1: 1.0000	Runtime: 817.9
Train Epoch: 12 [129600/269776 (60%)]	Loss: 0.065549	F1: 1.0000	Runtime: 828.3
Train Epoch: 12 [131200/269776 (61%)]	Loss: 0.507983	F1: 0.8750	Runtime: 838.7
Train Epoch: 12 [132800/269776 (62%)]	Loss: 0.002697	F1: 1.0000	Runtime: 849.2
Train Epoch: 12 [134400/269776 (62%)]	Loss: 0.223120	F1: 0.7500	Runtime: 859.6
Train Epoch: 12 [136000/269776 (63%)]	Loss: 0.008152	F1: 1.0000	Runtime: 870.1
Train Epoch: 12 [137600/269776 (64%)]	Loss: 0.829671

Train Epoch: 13 [68800/269776 (32%)]	Loss: 0.076757	F1: 1.0000	Runtime: 433.9
Train Epoch: 13 [70400/269776 (33%)]	Loss: 0.000822	F1: 1.0000	Runtime: 444.0
Train Epoch: 13 [72000/269776 (33%)]	Loss: 0.000457	F1: 1.0000	Runtime: 454.1
Train Epoch: 13 [73600/269776 (34%)]	Loss: 0.005065	F1: 1.0000	Runtime: 464.3
Train Epoch: 13 [75200/269776 (35%)]	Loss: 0.008091	F1: 1.0000	Runtime: 474.4
Train Epoch: 13 [76800/269776 (36%)]	Loss: 0.002637	F1: 1.0000	Runtime: 484.7
Train Epoch: 13 [78400/269776 (36%)]	Loss: 0.022537	F1: 1.0000	Runtime: 494.9
Train Epoch: 13 [80000/269776 (37%)]	Loss: 0.031138	F1: 1.0000	Runtime: 505.1
Train Epoch: 13 [81600/269776 (38%)]	Loss: 0.012779	F1: 1.0000	Runtime: 515.4
Train Epoch: 13 [83200/269776 (39%)]	Loss: 0.000516	F1: 1.0000	Runtime: 525.6
Train Epoch: 13 [84800/269776 (39%)]	Loss: 0.110378	F1: 1.0000	Runtime: 535.7
Train Epoch: 13 [86400/269776 (40%)]	Loss: 0.070004	F1: 1.0000	Runtime: 545.9
Train Epoch: 13 [88000/269776 (41%)]	Loss: 0.001822	F1: 1.0000	R

Train Epoch: 14 [17600/269776 (8%)]	Loss: 0.009521	F1: 1.0000	Runtime: 110.3
Train Epoch: 14 [19200/269776 (9%)]	Loss: 0.004530	F1: 1.0000	Runtime: 120.4
Train Epoch: 14 [20800/269776 (10%)]	Loss: 0.023784	F1: 1.0000	Runtime: 130.4
Train Epoch: 14 [22400/269776 (10%)]	Loss: 0.009084	F1: 1.0000	Runtime: 140.4
Train Epoch: 14 [24000/269776 (11%)]	Loss: 0.072013	F1: 1.0000	Runtime: 150.5
Train Epoch: 14 [25600/269776 (12%)]	Loss: 0.014865	F1: 1.0000	Runtime: 160.5
Train Epoch: 14 [27200/269776 (13%)]	Loss: 0.166228	F1: 0.8750	Runtime: 170.7
Train Epoch: 14 [28800/269776 (13%)]	Loss: 0.001312	F1: 1.0000	Runtime: 180.5
Train Epoch: 14 [30400/269776 (14%)]	Loss: 0.048350	F1: 1.0000	Runtime: 190.7
Train Epoch: 14 [32000/269776 (15%)]	Loss: 0.090150	F1: 1.0000	Runtime: 200.7
Train Epoch: 14 [33600/269776 (16%)]	Loss: 0.002650	F1: 1.0000	Runtime: 210.8
Train Epoch: 14 [35200/269776 (16%)]	Loss: 0.032304	F1: 1.0000	Runtime: 221.0
Train Epoch: 14 [36800/269776 (17%)]	Loss: 0.004014	F1: 1.0000	Run

...

Train Epoch: 17 [38400/269776 (18%)]	Loss: 0.006034	F1: 1.0000	Runtime: 242.2
Train Epoch: 17 [40000/269776 (19%)]	Loss: 0.004151	F1: 1.0000	Runtime: 252.2
Train Epoch: 17 [41600/269776 (19%)]	Loss: 0.004538	F1: 1.0000	Runtime: 262.3
Train Epoch: 17 [43200/269776 (20%)]	Loss: 0.001944	F1: 1.0000	Runtime: 272.7
Train Epoch: 17 [44800/269776 (21%)]	Loss: 0.001338	F1: 1.0000	Runtime: 282.8
Train Epoch: 17 [46400/269776 (21%)]	Loss: 0.000346	F1: 1.0000	Runtime: 293.1
Train Epoch: 17 [48000/269776 (22%)]	Loss: 0.002888	F1: 1.0000	Runtime: 303.2
Train Epoch: 17 [49600/269776 (23%)]	Loss: 0.002449	F1: 1.0000	Runtime: 313.4
Train Epoch: 17 [51200/269776 (24%)]	Loss: 0.007086	F1: 1.0000	Runtime: 323.5
Train Epoch: 17 [52800/269776 (24%)]	Loss: 0.005569	F1: 1.0000	Runtime: 333.7
Train Epoch: 17 [54400/269776 (25%)]	Loss: 0.006732	F1: 1.0000	Runtime: 343.8
Train Epoch: 17 [56000/269776 (26%)]	Loss: 0.000932	F1: 1.0000	Runtime: 354.0
Train Epoch: 17 [57600/269776 (27%)]	Loss: 0.021552	F1: 1.0000	R

Train Epoch: 17 [204800/269776 (95%)]	Loss: 1.287204	F1: 0.7500	Runtime: 1316.8
Train Epoch: 17 [206400/269776 (96%)]	Loss: 0.000775	F1: 1.0000	Runtime: 1327.4
Train Epoch: 17 [208000/269776 (96%)]	Loss: 0.004421	F1: 1.0000	Runtime: 1338.0
Train Epoch: 17 [209600/269776 (97%)]	Loss: 0.001940	F1: 1.0000	Runtime: 1348.5
Train Epoch: 17 [211200/269776 (98%)]	Loss: 0.065623	F1: 1.0000	Runtime: 1359.0
Train Epoch: 17 [212800/269776 (99%)]	Loss: 0.013130	F1: 1.0000	Runtime: 1369.6
Train Epoch: 17 [214400/269776 (99%)]	Loss: 0.004638	F1: 1.0000	Runtime: 1380.1
after epoch 17 got f1 score of 0.9241104521868475
Train Epoch: 18 [0/269776 (0%)]	Loss: 0.017343	F1: 1.0000	Runtime: 0.1
Train Epoch: 18 [1600/269776 (1%)]	Loss: 0.010858	F1: 1.0000	Runtime: 10.1
Train Epoch: 18 [3200/269776 (1%)]	Loss: 0.022961	F1: 1.0000	Runtime: 20.1
Train Epoch: 18 [4800/269776 (2%)]	Loss: 0.000232	F1: 1.0000	Runtime: 30.2
Train Epoch: 18 [6400/269776 (3%)]	Loss: 0.265412	F1: 0.8750	Runtime: 40.3
Train Epoch: 18 [80

Train Epoch: 18 [156800/269776 (73%)]	Loss: 0.000113	F1: 1.0000	Runtime: 1002.0
Train Epoch: 18 [158400/269776 (73%)]	Loss: 0.005707	F1: 1.0000	Runtime: 1012.3
Train Epoch: 18 [160000/269776 (74%)]	Loss: 0.324263	F1: 0.8750	Runtime: 1022.7
Train Epoch: 18 [161600/269776 (75%)]	Loss: 0.190649	F1: 1.0000	Runtime: 1033.1
Train Epoch: 18 [163200/269776 (76%)]	Loss: 0.007489	F1: 1.0000	Runtime: 1043.5
Train Epoch: 18 [164800/269776 (76%)]	Loss: 0.001218	F1: 1.0000	Runtime: 1053.9
Train Epoch: 18 [166400/269776 (77%)]	Loss: 0.003791	F1: 1.0000	Runtime: 1064.3
Train Epoch: 18 [168000/269776 (78%)]	Loss: 0.001291	F1: 1.0000	Runtime: 1074.7
Train Epoch: 18 [169600/269776 (79%)]	Loss: 0.141057	F1: 1.0000	Runtime: 1085.1
Train Epoch: 18 [171200/269776 (79%)]	Loss: 0.036488	F1: 1.0000	Runtime: 1095.6
Train Epoch: 18 [172800/269776 (80%)]	Loss: 0.000542	F1: 1.0000	Runtime: 1106.0
Train Epoch: 18 [174400/269776 (81%)]	Loss: 0.003345	F1: 1.0000	Runtime: 1116.6
Train Epoch: 18 [176000/269776 (82%)]	Lo

Train Epoch: 19 [107200/269776 (50%)]	Loss: 0.012611	F1: 1.0000	Runtime: 681.6
Train Epoch: 19 [108800/269776 (50%)]	Loss: 0.088999	F1: 1.0000	Runtime: 691.9
Train Epoch: 19 [110400/269776 (51%)]	Loss: 0.001804	F1: 1.0000	Runtime: 702.2
Train Epoch: 19 [112000/269776 (52%)]	Loss: 0.030261	F1: 1.0000	Runtime: 712.6
Train Epoch: 19 [113600/269776 (53%)]	Loss: 0.015083	F1: 1.0000	Runtime: 722.9
Train Epoch: 19 [115200/269776 (53%)]	Loss: 0.032214	F1: 1.0000	Runtime: 733.2
Train Epoch: 19 [116800/269776 (54%)]	Loss: 0.175669	F1: 0.8750	Runtime: 743.6
Train Epoch: 19 [118400/269776 (55%)]	Loss: 0.061617	F1: 1.0000	Runtime: 754.0
Train Epoch: 19 [120000/269776 (56%)]	Loss: 0.000191	F1: 1.0000	Runtime: 764.7
Train Epoch: 19 [121600/269776 (56%)]	Loss: 0.013138	F1: 1.0000	Runtime: 775.1
Train Epoch: 19 [123200/269776 (57%)]	Loss: 0.103284	F1: 1.0000	Runtime: 785.4
Train Epoch: 19 [124800/269776 (58%)]	Loss: 0.003251	F1: 1.0000	Runtime: 795.8
Train Epoch: 19 [126400/269776 (59%)]	Loss: 0.000827

Train Epoch: 20 [57600/269776 (27%)]	Loss: 0.001920	F1: 1.0000	Runtime: 364.5
Train Epoch: 20 [59200/269776 (27%)]	Loss: 0.040862	F1: 1.0000	Runtime: 374.7
Train Epoch: 20 [60800/269776 (28%)]	Loss: 0.012797	F1: 1.0000	Runtime: 384.9
Train Epoch: 20 [62400/269776 (29%)]	Loss: 0.001577	F1: 1.0000	Runtime: 395.1
Train Epoch: 20 [64000/269776 (30%)]	Loss: 0.009621	F1: 1.0000	Runtime: 405.4
Train Epoch: 20 [65600/269776 (30%)]	Loss: 0.001769	F1: 1.0000	Runtime: 415.6
Train Epoch: 20 [67200/269776 (31%)]	Loss: 0.005005	F1: 1.0000	Runtime: 425.9
Train Epoch: 20 [68800/269776 (32%)]	Loss: 0.000988	F1: 1.0000	Runtime: 436.1
Train Epoch: 20 [70400/269776 (33%)]	Loss: 0.051815	F1: 1.0000	Runtime: 446.2
Train Epoch: 20 [72000/269776 (33%)]	Loss: 0.070148	F1: 1.0000	Runtime: 456.2
Train Epoch: 20 [73600/269776 (34%)]	Loss: 0.034420	F1: 1.0000	Runtime: 466.3
Train Epoch: 20 [75200/269776 (35%)]	Loss: 0.100235	F1: 1.0000	Runtime: 476.4
Train Epoch: 20 [76800/269776 (36%)]	Loss: 0.001641	F1: 1.0000	R

### save or load outputs

In [26]:
save_output(args, model, device, test_loader, 'MSResNet-score701', 
            trainDataset, testDataset, path_save = path_submission)

saved outputs


In [27]:
save_geometric_mean_predictions(path_submission + 'output-MSResNet-score701.txt', 
                                path_submission + 'output-SpectralResNet-score756.txt', 
                                path_submission, 
                                trainDataset, testDataset)

saved predictions


### Load Model

In [3]:
if load_model:
    #model = torch.load(path_model)
    model = torch.load('models/MSResNet_10epoch_score701.pt')

In [4]:
# load indices
with open(path_submission + "indices_MSResNet_score701.pt", "rb") as fp:
    indices = pickle.load(fp)

In [23]:
#get_mean_F1(model, validation_loader)
get_loss(model, validation_loader,device)

(2.308869604184242, 0.03617494440326128)

In [ ]:
epoch=10
test(args, model, device, test_loader, epoch, trainDataset, testDataset, path_submission)